### Read training, dev and unlabeled test data

The following provides a starting code (Python 3) of how to read the labeled training and dev cipher text, and unlabeled test cipher text, into lists.

In [1]:
train, dev, test = [], [], []

In [2]:
for x in open('./train_enc.tsv', encoding='utf-8'):
    x = x.rstrip('\n\r').split('\t')
    # x[0] will be the label (0 or 1), and x[1] will be the ciphertext sentence.
    x[0] = int(x[0]) 
    #print(x)
    train.append(x)
print (len(train))
print (train[:3])

16220
[[0, 'lkêcê yoúc cêêö y#êjl lw mówám Újám j Úêê# ütlk Úol lkêú z#ê ctöé8ú ówl xoóóú éê#xw#öê#c .'], [0, '6êcétlê jolêot8 zc éê#xw#öjóáê , tl zc j #jlkê# 8tcl8êcc jöÚ8ê 6wüó lkê öt668ê wx lkê #wj6 , ükê#ê lkê lkêöjltá t#wótêc j#ê lww wÚ2twoc jó6 lkê cê+oj8 éw8tltác lww cöoy .'], [0, 'tx lktc kw8t6jú öw2tê tc coééwcê6 lw Úê j ytxl , cwöêÚw6ú oóü#jééê6 tl êj#8ú , lwwm wol j88 lkê yww6 cloxx , jó6 8êxl Úêktó6 lkê á#jé ( 8tlê#j88ú ) .']]


In [3]:
for x in open('./dev_enc.tsv', encoding='utf-8'):
    x = x.rstrip('\n\r').split('\t')
    # x[0] will be the label (0 or 1), and x[1] will be the ciphertext sentence.
    x[0] = int(x[0]) 
    dev.append(x)
print (len(dev))
print (dev[:3])

2027
[[1, 'ów8jó Ú#j2ê8ú l#êj6c ükê#ê xêü jöê#tájó xt8öc 6j#ê lw 6ê82ê 77 tólw lkê üw#86 wx jöÚt2j8êóáê jó6 jöÚtyotlú <<<'], [0, 'ê2êó öo#ékú zc ê+éê#l áwötá ltötóy jó6 xjöê6 ákj#tcöj áj ózl #êcáoê lktc êxxw#l .'], [1, 'üt88 jcco#ê68ú #jóm jc wóê wx lkê á8ê2ê#êcl , öwcl 6êáêélt2ê8ú jöoctóy áwöê6têc wx lkê úêj# .']]


#### Different from 'train' and 'dev' that are both list of tuples, 'test' will be just a list.

In [4]:
for x in open('./test_enc_unlabeled.tsv', encoding='utf-8'):
    x = x.rstrip('\n\r')
    test.append(x)
print (len(test))
print (test[:3])

2028
['j 6t6jáltá jó6 6o88 6wáoöêólj#ú y8w#txútóy cwxlüj#ê jój#ákú .', 'ówlktóy cltámc , #êj88ú , ê+áêél j 8tóyê#tóy á#êêétóêcc wóê xêê8c x#wö Úêtóy 6#jyyê6 lk#woyk j cj6 , cw#6t6 oót2ê#cê wx yoóc , 6#oyc , j2j#táê jó6 6jöjyê6 6#êjöc .', 'öo#ékú jó6 üt8cwó jáloj88ú öjmê j é#êllú yww6 lêjö <<< Úol lkê é#wvêál co##woó6tóy lkêö tc 6tcl#êcctóy8ú #wlê .']


#### You can split every sentence into lists of words by white spaces.

In [5]:
train_split = [[x[0], x[1].split(' ')] for x in train]
dev_split = [[x[0], x[1].split(' ')] for x in dev]
test_split = [[x.split(' ')] for x in test]

### Main Code Body

You may choose to experiment with different methods using your program. However, you need to embed the training and inference processes at here. We will use your prediction on the unlabeled test data to grade, while checking this part to understand how your method has produced the predictions.

In [6]:
#Convert to DataFrame

import pandas as pd
train_frame = pd.DataFrame(train, columns = ['class', 'text'])
dev_frame = pd.DataFrame(dev, columns = ['class', 'text'])
test_frame = pd.DataFrame(test, columns = ['text'])

In [7]:
#Training Word Embdding 

from gensim.models.fasttext import FastText
word_token = []
for word_list in train_split:
  word_token.append(word_list[1])

embedding_size = 300
window_size = 5
down_sampling = 1e-2


fast_Text_model = FastText(word_token,
                      size=embedding_size,
                      window=window_size,
                      sample=down_sampling,
                      workers = 4,
                      sg=1,
                      iter=100)


In [8]:
#Vocab
vocab_words = list(fast_Text_model.wv.vocab)
vocab_size = len(vocab_words) +1
print(len(vocab_words))

6080


In [9]:
#saving embedding

filename = 'embedding_fasttxt.txt'
fast_Text_model.wv.save_word2vec_format(filename, binary=False)

In [10]:
#Text Processing

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words = vocab_size, filters='",.;@^_`{|}~\t\n' ) #; ` exist in dataset
tokenizer.fit_on_texts(train_frame['text'].values)

x_train = tokenizer.texts_to_sequences(train_frame['text'].values)
x_train = pad_sequences(x_train, padding = 'post' ,maxlen=65)
y_train = pd.get_dummies(train_frame['class']).values


x_d = tokenizer.texts_to_sequences(dev_frame['text'].values)
x_d = pad_sequences(x_d, padding = 'post' ,maxlen=65)
y_d = pd.get_dummies(dev_frame['class']).values


In [11]:
#Creating Embedding Matrix
from tqdm import tqdm
import numpy as np

embedding_vector = {}
f = open('embedding_fasttxt.txt')
for line in tqdm(f):
    value = line.split(' ')
    word = value[0]
    coef = np.array(value[1:],dtype = 'float32')
    embedding_vector[word] = coef

embedding_matrix = np.zeros((vocab_size,300))
for word,i in tqdm(tokenizer.word_index.items()):
    #print(word,i)
    embedding_value = embedding_vector.get(word)
    if embedding_value is not None:
        embedding_matrix[i] = embedding_value

6081it [00:00, 11951.56it/s]
100%|██████████| 20826/20826 [00:00<00:00, 841689.87it/s]


In [12]:
# from sklearn.model_selection import train_test_split
# x_dev, x_test, y_dev, y_test = train_test_split(x_d,y_d, test_size = 0.5, random_state = 42)


In [13]:
embedding_matrix.shape

(6081, 300)

In [14]:
import tensorflow as tf
import keras
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D, Bidirectional, Dropout

In [42]:
embid_dim = 300
lstm_out = 128

model = keras.Sequential()
model.add(Embedding(vocab_size, embid_dim, input_length =x_train.shape[1], weights = [embedding_matrix]))
model.add(Bidirectional(LSTM(lstm_out, dropout=0.2)))
model.add(Dense(128, activation = 'relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation = 'relu'))
model.add(Dense(2, activation = 'softmax'))
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 65, 300)           1824300   
                                                                 
 bidirectional_3 (Bidirectio  (None, 256)              439296    
 nal)                                                            
                                                                 
 dense_9 (Dense)             (None, 128)               32896     
                                                                 
 dropout_3 (Dropout)         (None, 128)               0         
                                                                 
 dense_10 (Dense)            (None, 64)                8256      
                                                                 
 dense_11 (Dense)            (None, 2)                 130       
                                                      

In [43]:
batch_size = 128
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
history = model.fit(x_train, y_train, epochs = 8, batch_size=batch_size, verbose = 1, validation_data =(x_d, y_d))

Epoch 1/8
127/127 [==============================] - 114s 863ms/step - loss: 0.5375 - accuracy: 0.7218 - val_loss: 0.4033 - val_accuracy: 0.8189
Epoch 2/8
127/127 [==============================] - 102s 804ms/step - loss: 0.3777 - accuracy: 0.8334 - val_loss: 0.3617 - val_accuracy: 0.8451
Epoch 3/8
127/127 [==============================] - 102s 802ms/step - loss: 0.3011 - accuracy: 0.8724 - val_loss: 0.3713 - val_accuracy: 0.8367
Epoch 4/8
127/127 [==============================] - 102s 805ms/step - loss: 0.2485 - accuracy: 0.8956 - val_loss: 0.3588 - val_accuracy: 0.8594
Epoch 5/8
127/127 [==============================] - 102s 799ms/step - loss: 0.2026 - accuracy: 0.9182 - val_loss: 0.3823 - val_accuracy: 0.8673
Epoch 6/8
127/127 [==============================] - 101s 792ms/step - loss: 0.1695 - accuracy: 0.9301 - val_loss: 0.3958 - val_accuracy: 0.8796
Epoch 7/8
127/127 [==============================] - 101s 795ms/step - loss: 0.1389 - accuracy: 0.9432 - val_loss: 0.3719 - val_ac

In [46]:
#Printing FINAL OUTPUT
X_test = tokenizer.texts_to_sequences(test_frame['text'].values)
X_test = pad_sequences(X_test, padding = 'post' ,maxlen=65)
y_pred = model.predict(X_test, verbose=1)
y_pred_final = np.argmax(y_pred, axis=1)

64/64 [==============================] - 6s 72ms/step


In [27]:
# y_pred = model.predict(x_test, verbose=1)

# y_test_f = np.argmax(y_test, axis=1)
# print(y_test_f)


# y_pred_f = np.argmax(y_pred, axis=1)
# print(y_pred_f)

# from sklearn.metrics import confusion_matrix
# from sklearn.metrics import accuracy_score
# print(confusion_matrix(y_test_f, y_pred_f))
# print(accuracy_score(y_test_f, y_pred_f))

32/32 [==============================] - 3s 76ms/step
[1 0 1 ... 0 1 1]
[1 0 1 ... 0 1 0]
[[407  65]
 [ 62 480]]
0.8747534516765286


In [ ]:
import keras
from matplotlib import pyplot as plt
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [58]:
# Eventually, results need to be a list of 2028 0 or 1's
results = []
results = y_pred_final

### Output Prediction Result File

You will need to submit a prediction result file. It should have 2028 lines, every line should be either 0 or 1, which is your model's prediction on the respective test set instance.

In [59]:
# suppose you had your model's predictions on the 2028 test cases read from test_enc_unlabeled.tsv, and 
#those results are in the list called 'results'
assert (len(results) == 2028)

In [60]:
# make sure the results are not float numbers, but intergers 0 and 1
results = [int(x) for x in results]

In [65]:
# write your prediction results to 'upload_predictions.txt' and upload that later
with open('upload_predictions.txt', 'w', encoding = 'utf-8') as fp:
    for x in results:
        fp.write(str(x) + '\n')